Ознайомитися з основними типами структур даних в Pandas (Series, DataFrame).
Навчитися обробляти та аналізувати дані за допомогою Pandas.
Опанувати базові методи візуалізації даних за допомогою Matplotlib та Seaborn.
Завантаження даних:

Ознайомтеся з доступними наборами даних в бібліотеці Seaborn (sns.load_dataset()).
Завантажте набір даних за власним вибором. Це може бути CSV файл, дані з веб-сайту або інший формат. Якщо не вдалося вибрати потрібний набір даних, то використайте "titanic" dataset - набір даних за замовчуванням.
Використовуйте pd.read_csv() для завантаження CSV файлів або sns.load_dataset() для завантаження вбудованих.

In [198]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [199]:
df = pd.read_csv("./datasets/games.csv")

# Обробка та аналіз даних у Pandas

## Створення DataFrame  
- `df = pd.read_csv('filename.csv')` – Завантаження даних з CSV-файлу.

## Перегляд перших та останніх рядків DataFrame  
- `df.head()` – Перші 5 рядків.  
- `df.tail()` – Останні 5 рядків.  

## Отримання інформації про структуру DataFrame  
- `df.info()` – Загальна інформація про DataFrame.  

## Виведення описової статистики для числових стовпців  
- `df.describe()` – Основні статистичні характеристики.  

## Вибір окремого стовпця або кількох стовпців  
- `df['column_name']` – Один стовпець.  
- `df[['col1', 'col2']]` – Декілька стовпців.  

## Фільтрація даних за умовою  
- `df[df['column_name'] > value]` – Відбір рядків за критерієм.  

## Сортування даних за певним стовпцем  
- `df.sort_values('column_name')` – Сортування за значеннями у стовпці.  

## Групування даних та обчислення агрегатних функцій  
- `df.groupby('category')['column_name'].mean()` – Обчислення середнього значення для кожної групи.  

## Додавання нового стовпця  
- `df['new_column'] = ...` – Додавання нового стовпця.  

## Видалення стовпця  
- `df.drop('column_name', axis=1)` – Видалення стовпця.  

## Перейменування стовпців  
- `df.rename(columns={'old_name': 'new_name'})` – Зміна назв стовпців.  


In [200]:
print(df.head())
print(df.tail())

                              Title Original Price Discounted Price  \
0                   Baldur's Gate 3         $29.99           $29.99   
1  Counter-Strike: Global Offensive         $14.99           $14.99   
2                     Apex Legends™           Free             Free   
3                   Forza Horizon 5         $34.78           $17.39   
4                     Call of Duty®           Free             Free   

   Release Date                                               Link  \
0   3 Aug, 2023  https://store.steampowered.com/app/1086940/Bal...   
1  21 Aug, 2012  https://store.steampowered.com/app/730/Counter...   
2   4 Nov, 2020  https://store.steampowered.com/app/1172470/Ape...   
3   8 Nov, 2021  https://store.steampowered.com/app/1551360/For...   
4  27 Oct, 2022  https://store.steampowered.com/app/1938090/Cal...   

                                    Game Description   Recent Reviews Summary  \
0  Baldur’s Gate 3 is a story-rich, party-based R...  Overwhelmingly Po

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71700 entries, 0 to 71699
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Title                   71699 non-null  object
 1   Original Price          71700 non-null  object
 2   Discounted Price        71700 non-null  object
 3   Release Date            71569 non-null  object
 4   Link                    71700 non-null  object
 5   Game Description        71489 non-null  object
 6   Recent Reviews Summary  56480 non-null  object
 7   All Reviews Summary     5371 non-null   object
 8   Recent Reviews Number   56480 non-null  object
 9   All Reviews Number      5371 non-null   object
 10  Developer               71479 non-null  object
 11  Publisher               71183 non-null  object
 12  Supported Languages     71700 non-null  object
 13  Popular Tags            71700 non-null  object
 14  Game Features           71700 non-null  object
 15  Mi

In [202]:
df.describe()

,Title,Original Price,Discounted Price,Release Date,Link,Game Description,Recent Reviews Summary,All Reviews Summary,Recent Reviews Number,All Reviews Number,Developer,Publisher,Supported Languages,Popular Tags,Game Features,Minimum Requirements
count,71699,71700,71700,71569,71700,71489,56480,5371,56480,5371,71479,71183,71700,71700,71700,70576
unique,71699,654,827,4441,71700,71090,18,7,13391,5296,45353,38543,11058,65817,5418,63201
top,Pixel Gun 3D: PC Edition,Free,Free,Coming soon,https://store.steampowered.com/app/2524890/Pix...,Find the objects that are hidden on the map.,Very Positive,Very Positive,- Need more user reviews to generate a score,- 90% of the 624 user reviews for this game ar...,Choice of Games,Big Fish Games,['English'],"['Indie', 'Casual']",['Single-player'],Requires a 64-bit processor and operating syst...
freq,1,17585,17585,5887,1,34,11039,3151,18999,3,162,459,38083,284,17262,274


In [203]:
df[["Title", "Game Description"]]

,Title,Game Description
0,Baldur's Gate 3,"Baldur’s Gate 3 is a story-rich, party-based R..."
1,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive (CS: GO) expa...
2,Apex Legends™,"Apex Legends is the award-winning, free-to-pla..."
3,Forza Horizon 5,Your Ultimate Horizon Adventure awaits! Explor...
4,Call of Duty®,"Welcome to Call of Duty® HQ, the home of Call ..."
...,...,...
71695,The Front,The Front is a survival-open-world-crafting (S...
71696,The Bornless,"Welcome to The Bornless, an action horror FPS...."
71697,Winter Memories,After the unforgettable summer memories you’ve...
71698,Call of Duty®: Modern Warfare® III,In the direct sequel to the record-breaking Ca...


In [204]:
free_games = df[df["Original Price"] == "Free"]["Title"]
free_games

2                   Apex Legends™
4                   Call of Duty®
5             PUBG: BATTLEGROUNDS
9              NARAKA: BLADEPOINT
15                    War Thunder
                   ...           
71694                  Sunkenland
71695                   The Front
71696                The Bornless
71697             Winter Memories
71699    Pixel Gun 3D: PC Edition
Name: Title, Length: 17585, dtype: object

In [205]:
print(f"There is {free_games.count()} game free to play in dataset")

There is 17584 game free to play in dataset


In [206]:
df["Original Price"] = (
    df["Original Price"]
    .str.replace("$", "")  
    .str.replace(",", "") 
    .replace("Free", 0)  
    .astype(float) 
)

df.sort_values("Original Price", ascending=False)

,Title,Original Price,Discounted Price,Release Date,Link,Game Description,Recent Reviews Summary,All Reviews Summary,Recent Reviews Number,All Reviews Number,Developer,Publisher,Supported Languages,Popular Tags,Game Features,Minimum Requirements
52064,Jack-O-Lantern Covers of Darkness,14046333.0,"$4,213,899.90","14 Dec, 2019",https://store.steampowered.com/app/1193660/Jac...,The Jack-O-Lanterns have emerged from the unde...,3 user reviews,NaN,- Need more user reviews to generate a score,NaN,"Strawberry Game Studios, LLC","Strawberry Game Studios, LLC",['English'],"['Casual', 'Adventure', 'Indie', 'First-Person...",['Single-player'],OS: | Windows 7/8/10 | Processor: | Intel® C...
60721,Fright,14046333.0,"$4,213,899.90","5 Jan, 2020",https://store.steampowered.com/app/1200630/Fri...,Move around a hauntred maze trying to find boo...,1 user reviews,NaN,- Need more user reviews to generate a score,NaN,"Strawberry Game Studios, LLC","Strawberry Game Studios, LLC",['English'],"['Indie', 'Casual', 'First-Person', 'Singlepla...",['Single-player'],OS: | Windows 8/8.1/10 | Processor: | Intel(...
52065,Roller,14046333.0,"$4,213,899.90","18 May, 2018",https://store.steampowered.com/app/847510/Roll...,Let the colors hypnotize you in this fun fille...,NaN,NaN,NaN,NaN,Strawberry Game Studios,Strawberry Game Studios,['English'],['Casual'],['Single-player'],OS: | Windows 8 | Processor: | Intel(R) Core...
39897,Nomad Station,100000.0,"$100,000.00","15 Jul, 2023",https://store.steampowered.com/app/2386790/Nom...,"Become a master MI6 saboteur, and earn your na...",NaN,NaN,NaN,NaN,Derek Moritz,Derek Moritz,['English'],"['Space', 'Sci-fi', 'Action-Adventure', 'RPG',...","['Single-player', 'Steam Achievements', 'Track...",Requires a 64-bit processor and operating syst...
63158,Orpiment,9999.0,"$9,999.00","11 Oct, 2021",https://store.steampowered.com/app/1569370/Orp...,"Orpiment is a rogue-like game, based on a deck...",NaN,NaN,NaN,NaN,Yobiké,Yobiké,"['English', 'French']","['Card Battler', 'Roguelite', 'Real Time Tacti...",['Single-player'],OS: | Windows 7 (SP1+) | Processor: | 1.1 GH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71679,ProtoZED,0.0,Free,2024,https://store.steampowered.com/app/2360880/Pro...,Coop extraction looter-shooter with colony sim...,NaN,NaN,NaN,NaN,Scott H.F. Dunbar,Hello Friend Games,['English'],"['Survival', 'Zombies', 'Action', 'Co-op', 'Lo...","['Single-player', 'Online Co-op', 'LAN Co-op',...",OS: | Windows 8/10 64-bit | Processor: | Dua...
71680,Survival & Horror: Undead Space,0.0,Free,2023,https://store.steampowered.com/app/2360390/Sur...,"The events take place in the future, on resear...",NaN,NaN,NaN,NaN,PunkOFF,Honor Games,"['English', 'Russian']","['Action', 'Adventure', 'Space', 'Shooter', 'E...","['Single-player', 'Steam Achievements', 'Full ...","OS: | Windows 7, 8, 10 (64-bit) | Processor: ..."
71681,Zone.E,0.0,Free,To be announced,https://store.steampowered.com/app/2360550/Zon...,Zone.E is a unique survival game where players...,NaN,NaN,NaN,NaN,Animpic Studio,Animpic Studio,"['English', 'Italian', 'Russian', 'Armenian']","['Adventure', 'Action', 'Action-Adventure', 'S...","['Single-player', 'Online PvP', 'Online Co-op']",Requires a 64-bit processor and operating syst...
71682,Heralds of the Avirentes - Ch. 1 Wings of Change,0.0,Free,Q4 2023,https://store.steampowered.com/app/2360560/Her...,"Heralds of the Avirentes is a visual novel, wh...",NaN,NaN,NaN,NaN,TellerySpyro_GTX,TellerySpyro_GTX,"['English', 'Ukrainian']","['Visual Novel', 'Female Protagonist', 'Fantas...","['Single-player', 'Full controller support']",OS: | Windows 10 or newer | Processor: | Int...


There is 3 game with price $4,213,899.90 😓😓😓

In [207]:
df.groupby('Release Date')['Original Price'].mean()

Release Date
1 Apr, 2006        2.99000
1 Apr, 2008        4.12000
1 Apr, 2013        0.00000
1 Apr, 2014        5.89000
1 Apr, 2015        5.11625
                    ...   
Sep 2020           1.39000
September 2018     0.00000
September 2023     0.00000
September 2024     0.00000
To be announced    0.00000
Name: Original Price, Length: 4441, dtype: float64

In [208]:
import re

def extract_rating(text):
    """Extracts the percentage value from a string."""
    if isinstance(text, str): 
        result = re.search(r'(\d+)%', text)
        return int(result.group(1)) if result else None
    return None 

In [209]:
df["Game rating"] = df["All Reviews Number"].map(extract_rating)
df["Game rating"]

0        94.0
1        88.0
2        80.0
3        88.0
4        57.0
         ... 
71695     NaN
71696     NaN
71697     NaN
71698     NaN
71699     NaN
Name: Game rating, Length: 71700, dtype: float64

In [210]:
df = df.drop(columns=["Publisher"])  # Removes the 'Publisher' column
df.columns

Index(['Title', 'Original Price', 'Discounted Price', 'Release Date', 'Link',
       'Game Description', 'Recent Reviews Summary', 'All Reviews Summary',
       'Recent Reviews Number', 'All Reviews Number', 'Developer',
       'Supported Languages', 'Popular Tags', 'Game Features',
       'Minimum Requirements', 'Game rating'],
      dtype='object')

In [211]:
df = df.rename(columns={"All Reviews Number": "All Reviews"})
df["All Reviews"].head()

0    - 94% of the 188,617 user reviews for this gam...
1    - 88% of the 7,428,921 user reviews for this g...
2    - 80% of the 701,597 user reviews for this gam...
3    - 88% of the 123,162 user reviews for this gam...
4    - 57% of the 236,876 user reviews for this gam...
Name: All Reviews, dtype: object

Matplotlib: Базова візуалізація:

Побудуйте гістограму розподілу для одного з числових стовпців (plt.hist(df['column_name'])).
Створіть точковий графік для відображення залежності між двома числовими стовпцями (plt.scatter(df['col1'], df['col2'])).
Побудуйте стовпчикову діаграму для порівняння категорій за певним показником (plt.bar(df['category'], df['value'])).
Відобразіть кругову діаграму для відображення частки кожної категорії (plt.pie(df['value'], labels=df['category'])).
Додайте підписи до осей, заголовок графіку та легенду (plt.xlabel(), plt.ylabel(), plt.title(), plt.legend()).
Збережіть графік у файл (plt.savefig('plot.png')).

Seaborn: Розширена візуалізація:

Використовуйте Seaborn для створення більш інформативних та естетично привабливих графіків.
Побудуйте графік розподілу за допомогою sns.displot() або sns.kdeplot().
Відобразіть залежність між змінними за допомогою sns.relplot() або sns.scatterplot().
Використовуйте категорійні графіки (sns.catplot(), sns.boxplot(), sns.violinplot()) для порівняння категорій.
Побудуйте теплову карту для відображення кореляції між змінними (sns.heatmap()).
Використовуйте FacetGrid для побудови графіків за категоріями.
Застосуйте різні стилі Seaborn для покращення зовнішнього вигляду графіків (sns.set_style()).
